In [ ]:
!pip install wandb --upgrade

In [ ]:
import wandb
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import glob
import datetime

from wandb.keras import WandbCallback
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.utils import Sequence
from keras.layers import AveragePooling2D, Input, Conv2D
from sklearn.model_selection import StratifiedKFold
from skimage.io import imread,imshow
from skimage.transform import resize
from sklearn.utils import shuffle

%matplotlib inline
%reload_ext tensorboard

wandb.login()

In [ ]:
class MergedGenerators(Sequence):

    def __init__(self, batch_size, generators=[], sub_batch_size=[]):
        self.generators = generators
        self.sub_batch_size = sub_batch_size
        self.batch_size = batch_size

    def __len__(self):
        return int(
            sum([(len(self.generators[idx]) * self.sub_batch_size[idx])
                 for idx in range(len(self.sub_batch_size))]) /
            self.batch_size)

    def __getitem__(self, index):
        X_batch = []
        Y_batch = []
        for generator in self.generators:
            if generator.class_mode is None:
                x1 = generator[index % len(generator)]
                X_batch = [*X_batch, *x1]

            else:
                x1, y1 = generator[index % len(generator)]
                X_batch = [*X_batch, *x1]
                Y_batch = [*Y_batch, *y1]

        if self.generators[0].class_mode is None:
            return np.array(X_batch)
        return np.array(X_batch), np.array(Y_batch)


def build_datagenerator(dir1=None, dir2=None, dir3=None, batch_size=32):
    n_images_in_dir1 = sum([len(files) for r, d, files in os.walk(dir1)])
    n_images_in_dir2 = sum([len(files) for r, d, files in os.walk(dir2)])
    n_images_in_dir3 = sum([len(files) for r, d, files in os.walk(dir3)])

    generator1_batch_size = int((n_images_in_dir1 * batch_size) /
                                (n_images_in_dir1 + n_images_in_dir2 + n_images_in_dir3))
    
    generator2_batch_size = int((n_images_in_dir2 * batch_size) /
                                (n_images_in_dir1 + n_images_in_dir2 + n_images_in_dir3))

    generator3_batch_size = batch_size - generator1_batch_size - generator2_batch_size

    generator = ImageDataGenerator(rescale=1. / 255)

    generator1 = generator.flow_from_directory(
        dir1,
        target_size=(600, 600),
        color_mode='rgb',
        class_mode='binary',
        batch_size=generator1_batch_size,
        shuffle=True,
        seed=171)

    generator2 = generator.flow_from_directory(
        dir2,
        target_size=(600, 600),
        color_mode='rgb',
        class_mode='binary',
        batch_size=generator2_batch_size,
        shuffle=True,
        seed=171)
    
    generator3 = generator.flow_from_directory(
        dir3,
        target_size=(600, 600),
        color_mode='rgb',
        class_mode='binary',
        batch_size=generator3_batch_size,
        shuffle=True,
        seed=171)

    return MergedGenerators(
        batch_size,
        generators=[generator1, generator2, generator3],
        sub_batch_size=[generator1_batch_size, generator2_batch_size, generator2_batch_size])



In [ ]:
from keras.models import Model
from keras.layers import *
from keras.utils import plot_model

def MyModel():
    model = keras.Sequential()

    model.add(AveragePooling2D(input_shape=(600,600,3),
                               pool_size=(2, 2)))
    
    model.add(Conv2D(filters=wandb.config.first_conv_filters, 
                     kernel_size=(wandb.config.kernel_size, wandb.config.kernel_size), 
                     activation='relu'))
    
    model.add(AveragePooling2D())

    model.add(Conv2D(filters=wandb.config.second_conv_filters, 
                     kernel_size=(wandb.config.kernel_size, wandb.config.kernel_size),
                     activation='relu'))
    
    model.add(AveragePooling2D())

    model.add(Flatten())

    model.add(Dense(units=120, activation='relu'))
    
    model.add(Dense(units=84, activation='relu'))

    model.add(Dense(units=1, activation = 'sigmoid'))
    
    return model



In [ ]:
sweep_config = {
  'method': 'bayes', 
  'metric': {
      'name': 'val_loss',
      'goal': 'minimize'
  },
  'early_terminate':{
      'type': 'hyperband',
      'min_iter': 5
  },
  'parameters': {
      'batch_size': {
          'values': [8, 16, 32, 64]
      },
      'learning_rate':{
          'values': [1, 0.5, 0.1, 0.05, 0.01, 0.005, 0.001, 0.0005, 0.0001]
      },
      'first_conv_filters':{
          'values': [1,3,6,9]
      },
      'second_conv_filters':{
          'values': [1,3,6,9]
      },
      'kernel_size':{
          'values': [3,5,15]
      }
  }
}

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="LC-LeNet")
(sweep_id)

In [ ]:
sweep_id = 'romcp10n'

In [ ]:
wandb.agent(sweep_id, function=train)